In [25]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.metrics import balanced_accuracy_score

Reshape the Input Data

In [26]:
X_train_b = np.load("Xtrain2_b.npy")
Y_train_b = np.load("Ytrain2_b.npy")

X_test_b = np.load("Xtest2_b.npy")

Xtrain2_b_reshaped = X_train_b.reshape(547, 48, 48, 1)
Ytrain2_b_reshaped = Y_train_b.reshape(547, 48, 48, 1)

Ytrain2_b_reshaped = (Ytrain2_b_reshaped > 0).astype(np.float32)

X_train, X_val, y_train, y_val = train_test_split(Xtrain2_b_reshaped, Ytrain2_b_reshaped, test_size=0.2, random_state=42)

Define the FCN Model


In [ ]:
def create_fcn_binary_model(input_shape):
    inputs = keras.Input(shape=input_shape)

    # Encoder
    x = layers.Conv2D(32, kernel_size=3, activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D(pool_size=2)(x)
    
    x = layers.Conv2D(64, kernel_size=3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(pool_size=2)(x)

    x = layers.Conv2D(128, kernel_size=3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(pool_size=2)(x)

    # Bottleneck
    x = layers.Conv2D(256, kernel_size=3, activation='relu', padding='same')(x)

    # Decoder
    x = layers.Conv2DTranspose(128, kernel_size=3, activation='relu', padding='same')(x)
    x = layers.UpSampling2D(size=(2, 2))(x)
    
    x = layers.Conv2DTranspose(64, kernel_size=3, activation='relu', padding='same')(x)
    x = layers.UpSampling2D(size=(2, 2))(x)

    x = layers.Conv2DTranspose(32, kernel_size=3, activation='relu', padding='same')(x)
    x = layers.UpSampling2D(size=(2, 2))(x)

    outputs = layers.Conv2D(1, kernel_size=1, activation='sigmoid')(x) 

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


input_shape = (48, 48, 1)  
model = create_fcn_binary_model(input_shape)
model.summary()

Training the model

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_val, y_val), callbacks=[early_stopping])

Model Evaluation

In [ ]:
val_loss, val_accuracy = model.evaluate(X_val, y_val)

print(f'Validation Loss: {val_loss:.4f}')
print(f'Validation Accuracy: {val_accuracy:.4f}')

predictions = model.predict(X_val)

binary_predictions = (predictions > 0.5).astype(np.uint8)

y_val_flat = y_val.ravel()
binary_predictions_flat = binary_predictions.ravel()

balanced_acc = balanced_accuracy_score(y_val_flat, binary_predictions_flat)

print(f'Balanced Accuracy: {balanced_acc:.4f}')


def plot_images(original, ground_truth, predicted, n=5):
    plt.figure(figsize=(15, 5))
    for i in range(n):
        # Plot original images
        plt.subplot(3, n, i + 1)
        plt.imshow(original[i].reshape(48, 48), cmap='gray')
        plt.title('Original Image')
        plt.axis('off')

        # Plot ground truth
        plt.subplot(3, n, i + 1 + n)
        plt.imshow(ground_truth[i].reshape(48, 48), cmap='gray')
        plt.title('Ground Truth')
        plt.axis('off')

        # Plot predicted
        plt.subplot(3, n, i + 1 + 2*n)
        plt.imshow(predicted[i].reshape(48, 48), cmap='gray')
        plt.title('Predicted')
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

plot_images(X_val, y_val, binary_predictions)
